In [ ]:
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import itertools

In [ ]:
# all images will be converted to this size
ROWS = 256
COLS = 256
CHANNELS = 3

In [ ]:
train_image_generator = ImageDataGenerator(horizontal_flip=True, rescale=1./255, rotation_range=45)
test_image_generator = ImageDataGenerator(horizontal_flip=False, rescale=1./255, rotation_range=0)

train_generator = train_image_generator.flow_from_directory('trains', target_size=(ROWS, COLS), class_mode='categorical')
test_generator = test_image_generator.flow_from_directory('tests', target_size=(ROWS, COLS), class_mode='categorical')

In [ ]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# add a fully-connected layer
x = Dense(1024, activation='relu')(x)
out_layer = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=out_layer)

In [5]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                     32)                                                           

                                                                                                  
 average_pooling2d (AveragePool  (None, None, None,   0          ['max_pooling2d_1[0][0]']        
 ing2D)                         192)                                                              
                                                                                                  
 conv2d_5 (Conv2D)              (None, None, None,   12288       ['max_pooling2d_1[0][0]']        
                                64)                                                               
                                                                                                  
 conv2d_7 (Conv2D)              (None, None, None,   76800       ['activation_6[0][0]']           
                                64)                                                               
                                                                                                  
 conv2d_10

                                                                                                  
 conv2d_18 (Conv2D)             (None, None, None,   16384       ['average_pooling2d_1[0][0]']    
                                64)                                                               
                                                                                                  
 batch_normalization_12 (BatchN  (None, None, None,   192        ['conv2d_12[0][0]']              
 ormalization)                  64)                                                               
                                                                                                  
 batch_normalization_14 (BatchN  (None, None, None,   192        ['conv2d_14[0][0]']              
 ormalization)                  64)                                                               
                                                                                                  
 batch_nor

                                                                                                  
 batch_normalization_25 (BatchN  (None, None, None,   192        ['conv2d_25[0][0]']              
 ormalization)                  64)                                                               
                                                                                                  
 activation_19 (Activation)     (None, None, None,   0           ['batch_normalization_19[0][0]'] 
                                64)                                                               
                                                                                                  
 activation_21 (Activation)     (None, None, None,   0           ['batch_normalization_21[0][0]'] 
                                64)                                                               
                                                                                                  
 activatio

                                128)                                                              
                                                                                                  
 conv2d_36 (Conv2D)             (None, None, None,   114688      ['activation_35[0][0]']          
                                128)                                                              
                                                                                                  
 batch_normalization_31 (BatchN  (None, None, None,   384        ['conv2d_31[0][0]']              
 ormalization)                  128)                                                              
                                                                                                  
 batch_normalization_36 (BatchN  (None, None, None,   384        ['conv2d_36[0][0]']              
 ormalization)                  128)                                                              
          

...

 batch_normalization_53 (BatchN  (None, None, None,   576        ['conv2d_53[0][0]']              
 ormalization)                  192)                                                              
                                                                                                  
 batch_normalization_58 (BatchN  (None, None, None,   576        ['conv2d_58[0][0]']              
 ormalization)                  192)                                                              
                                                                                                  
 batch_normalization_59 (BatchN  (None, None, None,   576        ['conv2d_59[0][0]']              
 ormalization)                  192)                                                              
                                                                                                  
 activation_50 (Activation)     (None, None, None,   0           ['batch_normalization_50[0][0]'] 
          

 conv2d_60 (Conv2D)             (None, None, None,   147456      ['mixed6[0][0]']                 
                                192)                                                              
                                                                                                  
 conv2d_63 (Conv2D)             (None, None, None,   258048      ['activation_62[0][0]']          
                                192)                                                              
                                                                                                  
 conv2d_68 (Conv2D)             (None, None, None,   258048      ['activation_67[0][0]']          
                                192)                                                              
                                                                                                  
 conv2d_69 (Conv2D)             (None, None, None,   147456      ['average_pooling2d_6[0][0]']    
          

 batch_normalization_71 (BatchN  (None, None, None,   960        ['conv2d_71[0][0]']              
 ormalization)                  320)                                                              
                                                                                                  
 batch_normalization_75 (BatchN  (None, None, None,   576        ['conv2d_75[0][0]']              
 ormalization)                  192)                                                              
                                                                                                  
 activation_71 (Activation)     (None, None, None,   0           ['batch_normalization_71[0][0]'] 
                                320)                                                              
                                                                                                  
 activation_75 (Activation)     (None, None, None,   0           ['batch_normalization_75[0][0]'] 
          

                                384)                                                              
                                                                                                  
 activation_79 (Activation)     (None, None, None,   0           ['batch_normalization_79[0][0]'] 
                                384)                                                              
                                                                                                  
 activation_82 (Activation)     (None, None, None,   0           ['batch_normalization_82[0][0]'] 
                                384)                                                              
                                                                                                  
 activation_83 (Activation)     (None, None, None,   0           ['batch_normalization_83[0][0]'] 
                                384)                                                              
          

 ormalization)                  384)                                                              
                                                                                                  
 batch_normalization_92 (BatchN  (None, None, None,   1152       ['conv2d_92[0][0]']              
 ormalization)                  384)                                                              
                                                                                                  
 conv2d_93 (Conv2D)             (None, None, None,   393216      ['average_pooling2d_8[0][0]']    
                                192)                                                              
                                                                                                  
 batch_normalization_85 (BatchN  (None, None, None,   960        ['conv2d_85[0][0]']              
 ormalization)                  320)                                                              
          

In [6]:
tensorboard = TensorBoard(log_dir='./logs')

model.fit_generator(train_generator, steps_per_epoch=32, epochs=100, callbacks=[tensorboard], verbose=2)

C:\Users\FernandaAlmeidaRuas\AppData\Local\Temp\ipykernel_107968\1294760711.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=32, epochs=100, callbacks=[tensorboard], verbose=2)


Epoch 1/100
32/32 - 63s - loss: 5.6356 - accuracy: 0.0547 - 63s/epoch - 2s/step
Epoch 2/100
32/32 - 65s - loss: 4.5552 - accuracy: 0.0938 - 65s/epoch - 2s/step
Epoch 3/100
32/32 - 66s - loss: 3.9901 - accuracy: 0.1260 - 66s/epoch - 2s/step
Epoch 4/100
32/32 - 69s - loss: 3.5953 - accuracy: 0.1631 - 69s/epoch - 2s/step
Epoch 5/100
32/32 - 69s - loss: 3.3099 - accuracy: 0.1943 - 69s/epoch - 2s/step
Epoch 6/100
32/32 - 71s - loss: 3.1579 - accuracy: 0.2217 - 71s/epoch - 2s/step
Epoch 7/100
32/32 - 71s - loss: 2.8232 - accuracy: 0.2803 - 71s/epoch - 2s/step
Epoch 8/100
32/32 - 79s - loss: 2.7557 - accuracy: 0.3027 - 79s/epoch - 2s/step
Epoch 9/100
32/32 - 86s - loss: 2.6122 - accuracy: 0.3203 - 86s/epoch - 3s/step
Epoch 10/100
32/32 - 90s - loss: 2.6558 - accuracy: 0.3115 - 90s/epoch - 3s/step
Epoch 11/100
32/32 - 82s - loss: 2.5622 - accuracy: 0.3477 - 82s/epoch - 3s/step
Epoch 12/100
32/32 - 79s - loss: 2.5254 - accuracy: 0.3389 - 79s/epoch - 2s/step
Epoch 13/100
32/32 - 78s - loss: 2.44

In [7]:
print(model.evaluate_generator(test_generator, steps=5000))

C:\Users\FernandaAlmeidaRuas\AppData\Local\Temp\ipykernel_107968\2476778543.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  print(model.evaluate_generator(test_generator, steps=5000))


[1.242915391921997, 0.6441296339035034]


In [8]:
# unfreeze all layers for more training
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_generator, steps_per_epoch=32, epochs=100)

C:\Users\FernandaAlmeidaRuas\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\FernandaAlmeidaRuas\AppData\Local\Temp\ipykernel_107968\270955657.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=32, epochs=100)


Epoch 1/100
32/32 [==============================] - 352s 11s/step - loss: 3.0125 - accuracy: 0.3066
Epoch 2/100
32/32 [==============================] - 344s 11s/step - loss: 2.2891 - accuracy: 0.4297
Epoch 3/100
32/32 [==============================] - 327s 10s/step - loss: 1.9773 - accuracy: 0.4795
Epoch 4/100
32/32 [==============================] - 335s 10s/step - loss: 1.7022 - accuracy: 0.5156
Epoch 5/100
32/32 [==============================] - 328s 10s/step - loss: 1.6402 - accuracy: 0.5410
Epoch 6/100
32/32 [==============================] - 334s 10s/step - loss: 1.6296 - accuracy: 0.5576
Epoch 7/100
32/32 [==============================] - 334s 10s/step - loss: 1.5473 - accuracy: 0.5693
Epoch 8/100
32/32 [==============================] - 320s 10s/step - loss: 1.4764 - accuracy: 0.5977
Epoch 9/100
32/32 [==============================] - 339s 11s/step - loss: 1.4615 - accuracy: 0.6096
Epoch 10/100
32/32 [==============================] - 338s 11s/step - loss: 1.4135 - accura

Epoch 82/100
32/32 [==============================] - 278s 9s/step - loss: 0.6467 - accuracy: 0.8135
Epoch 83/100
32/32 [==============================] - 278s 9s/step - loss: 0.6165 - accuracy: 0.8135
Epoch 84/100
32/32 [==============================] - 272s 8s/step - loss: 0.6691 - accuracy: 0.7938
Epoch 85/100
32/32 [==============================] - 279s 9s/step - loss: 0.6025 - accuracy: 0.8154
Epoch 86/100
32/32 [==============================] - 286s 9s/step - loss: 0.6684 - accuracy: 0.7969
Epoch 87/100
32/32 [==============================] - 284s 9s/step - loss: 0.5517 - accuracy: 0.8516
Epoch 88/100
32/32 [==============================] - 288s 9s/step - loss: 0.6465 - accuracy: 0.8135
Epoch 89/100
32/32 [==============================] - 278s 9s/step - loss: 0.6089 - accuracy: 0.8193
Epoch 90/100
32/32 [==============================] - 277s 9s/step - loss: 0.6384 - accuracy: 0.8213
Epoch 91/100
32/32 [==============================] - 278s 9s/step - loss: 0.6666 - accurac

In [9]:
test_generator.reset()
print(model.evaluate_generator(test_generator, steps=5000))

C:\Users\FernandaAlmeidaRuas\AppData\Local\Temp\ipykernel_107968\2534557482.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  print(model.evaluate_generator(test_generator, steps=5000))


[0.4003739356994629, 0.8839497566223145]


In [10]:
model.save("birds-inceptionv3.model")

INFO:tensorflow:Assets written to: birds-inceptionv3.model\assets


INFO:tensorflow:Assets written to: birds-inceptionv3.model\assets
